# Indicadores de Trazabilidad 



## Importación de datos, librerias

In [1]:
# Importa librerias necesarias para el análisis

import pandas as pd
import numpy as np
import plotly.express as px

In [2]:
# Importa datos de población regional y crea DF


df_poblacion = pd.read_csv('data/aysen_population.csv') #Archivo ubicado dentro de la carpeta data

df_poblacion.head(2)

,Unnamed: 0,orden,nombre_region,codigo_region,nombre_provincia,codigo_provincia,nombre_comuna,codigo_comuna,edad_hombres,edad_mujeres,total
0,0,7172,Región de Aysén del General Carlos Ibáñez del ...,11,Coyhaique,111,Coyhaique,11101,28763,29055,57818
1,1,7194,Región de Aysén del General Carlos Ibáñez del ...,11,Coyhaique,111,Lago Verde,11102,494,358,852


In [3]:
# Importa datos de comunas corregidas por establecimiento de salud

df_establecimiento = pd.read_excel('data/establecimientos_salud_dict.xlsx') #Archivo ubicado dentro de la carpeta data


df_establecimiento.head()

,establecimiento_salud,comuna_seremi_xi,provincia_seremi_xi
0,Actividades gestionadas por la Dirección del S...,Coyhaique,Coyhaique
1,Centro Clínico Militar Coyhaique,Coyhaique,Coyhaique
2,Centro Comunitario de Salud Familiar Villa Ma...,Aysen,Aysen
3,Centro Comunitario de Salud Familiar Alejandro...,Coyhaique,Coyhaique
4,Centro Comunitario de Salud Familiar Puerto Ch...,Aysen,Aysen


In [4]:
# Importa datos desde la base de datos recivida via FTP
## PRIMERO: Guardar archivo dentro de la carpeta data
## SEGUNDO: No cambiar nombre ni formato del archivo recibido
## TERCERO: RECORDAR CAMBIAR FECHA EN EL ARCHIVO A IMPORTAR A CONTINUACION, i.e: 20210829 para este domingo 29 de Agosto 2021

df =  pd.read_csv('data/20210905_Región de Aysén del General Carlos Ibáñez del Campo_notificaciones_0600.csv', 
                   sep='~', # CSV con separador ''~''
                   low_memory=False
                  )

In [5]:
df_trazabilidad =  pd.read_csv('data/20210905_Región de Aysén del General Carlos Ibáñez del Campo_Seguimiento.csv', 
                   sep=';', # CSV con separador ''~''
                   low_memory=False
                  )

##  limpieza y ordenamiento 

In [6]:
# Une datos con diccionario comunal

dff = df.merge(df_establecimiento, how='left')

In [7]:
# Integra datos poblacionales

df_merged = dff.merge(df_poblacion.drop(columns= ['Unnamed: 0','orden','nombre_region', 'codigo_region', 'nombre_provincia','codigo_provincia']), left_on='comuna_seremi_xi', right_on='nombre_comuna', how='left' )

In [8]:
# Filtra datos obtenidos solo para pacientes con residencia en la Región de Aysén

data_aysen = df_merged[df_merged['region_residencia'] == 'Región de Aysén del General Carlos Ibáñez del Campo']

data_aysen.head(1)

,id_formulario_eno,numero_folio,id_enfermedad_eno,enfermedad_notificada,semana_epidemiologica,fecha_notificacion,etapa_clinica,establecimiento_salud,region,seremi,...,tac_torax_covid,condicion,region_separacion,comuna_seremi_xi,provincia_seremi_xi,nombre_comuna,codigo_comuna,edad_hombres,edad_mujeres,total
0,100166,50166,50,Coronavirus (COVID-19),11,2020-03-12,DESCARTADA,Hospital Regional (Coihaique),Región de Aysén del General Carlos Ibáñez del ...,SEREMI De Aisén del General Carlos Ibañez del ...,...,NaN,NaN,Región de Aysén del General Carlos Ibáñez del ...,Coyhaique,Coyhaique,Coyhaique,11101.0,28763.0,29055.0,57818.0


In [10]:
data_aysen_filtered = data_aysen[(data_aysen['estado_caso']!='No validada')&(
                          data_aysen['vigente_no_eliminado']==True)&(
                          data_aysen['seremi']=='SEREMI De Aisén del General Carlos Ibañez del Campo')
                       ].rename(columns={'edad_hombres':'pop_hombres',
                                         'edad_mujeres':'pop_mujeres',
                                         'total':'pop_total'})

data_aysen_filtered.sample(1)

,id_formulario_eno,numero_folio,id_enfermedad_eno,enfermedad_notificada,semana_epidemiologica,fecha_notificacion,etapa_clinica,establecimiento_salud,region,seremi,...,tac_torax_covid,condicion,region_separacion,comuna_seremi_xi,provincia_seremi_xi,nombre_comuna,codigo_comuna,pop_hombres,pop_mujeres,pop_total
37892,5930004,5880004,50,Coronavirus (COVID-19),52,2020-12-20,DESCARTADA,Hospital de Puerto Aysén,Región de Aysén del General Carlos Ibáñez del ...,SEREMI De Aisén del General Carlos Ibañez del ...,...,NO,NaN,Región de Aysén del General Carlos Ibáñez del ...,Aysen,Aysen,Aysen,11201.0,12719.0,11240.0,23959.0


In [11]:
# Transforma Fechas a formato DateTime 

data_aysen_filtered['fecha_notificacion'] = pd.to_datetime(data_aysen_filtered['fecha_notificacion'], errors = 'coerce') 

data_aysen_filtered['fecha_toma_muestra_1'] = pd.to_datetime(data_aysen_filtered['fecha_toma_muestra_1'], errors = 'coerce')

data_aysen_filtered['fecha_toma_muestra_2'] = pd.to_datetime(data_aysen_filtered['fecha_toma_muestra_2'], errors = 'coerce')

data_aysen_filtered['fecha_toma_muestra_3'] = pd.to_datetime(data_aysen_filtered['fecha_toma_muestra_3'], errors = 'coerce')

data_aysen_filtered['fecha_resultado_pcr_1'] = pd.to_datetime(data_aysen_filtered['fecha_resultado_pcr_1'], errors = 'coerce')

data_aysen_filtered['fecha_resultado_pcr_2'] = pd.to_datetime(data_aysen_filtered['fecha_resultado_pcr_2'], errors = 'coerce')

data_aysen_filtered['fecha_resultado_pcr_3'] = pd.to_datetime(data_aysen_filtered['fecha_resultado_pcr_3'], errors = 'coerce')

In [12]:
# Muestra variables presente en DB en formato DF

s=pd.DataFrame(data_aysen_filtered.columns)

s[s[0]=='tipo_seguimiento']


,0


In [13]:
# Selecciona variables de importancia

data_aysen_clean =  data_aysen_filtered[[
                                'numero_folio',
                                'comuna_residencia',
                                'fecha_notificacion',
                                'semana_epidemiologica',
                                'etapa_clinica',
                                'tipo_caso_busqueda',
                                'establecimiento_salud',
                                'resultado_pcr_1',
                                'resultado_pcr_2',
                                'resultado_pcr_3',
                                'fecha_toma_muestra_1',
                                'fecha_toma_muestra_2',
                                'fecha_toma_muestra_3',
                                'fecha_resultado_pcr_1',
                                'fecha_resultado_pcr_2',
                                'fecha_resultado_pcr_3',
                                'estado_paciente',
                                'lugar_reposo',
                                'estado_caso',
                                'presentacion_clinica',
                                'contacto_caso_sospechoso',
                                'motivo_examen',
                                'hospitalizacion',
                                'trabajador_salud',
                                'caso_cerrado',
                                'fecha_primeros_sintomas',
                                'fecha_diagnostico',
                                'comuna_seremi_xi',
                                'provincia_seremi_xi',
                                'sexo',
                                'edad',
                                'pop_hombres',
                                'pop_mujeres',
                                'pop_total'
                               ]]

In [14]:
# Agrega 53 semanas a las semanas de 2021 y crea semana_epidemiologica_21

data_aysen_clean1 = data_aysen_clean[data_aysen_clean['fecha_notificacion']<=pd.to_datetime('2021-01-02')]
data_aysen_clean2 = data_aysen_clean[data_aysen_clean['fecha_notificacion']>=pd.to_datetime('2021-01-03')]


data_aysen_clean1['semana_epidemiologica_21'] = data_aysen_clean1['semana_epidemiologica']
data_aysen_clean2['semana_epidemiologica_21'] = data_aysen_clean2['semana_epidemiologica']+53

data_aysen_clean_21 = data_aysen_clean1.append(data_aysen_clean2, ignore_index=True)


<ipython-input-14-d2f259f8832e>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_aysen_clean1['semana_epidemiologica_21'] = data_aysen_clean1['semana_epidemiologica']
<ipython-input-14-d2f259f8832e>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_aysen_clean2['semana_epidemiologica_21'] = data_aysen_clean2['semana_epidemiologica']+53


In [15]:
data_aysen_clean.sample(10)

,numero_folio,comuna_residencia,fecha_notificacion,semana_epidemiologica,etapa_clinica,tipo_caso_busqueda,establecimiento_salud,resultado_pcr_1,resultado_pcr_2,resultado_pcr_3,...,caso_cerrado,fecha_primeros_sintomas,fecha_diagnostico,comuna_seremi_xi,provincia_seremi_xi,sexo,edad,pop_hombres,pop_mujeres,pop_total
140092,15052008,Chile Chico,2021-07-18,29,DESCARTADA,Búsqueda activa,Hospital Dr. Leopoldo Ortega R. (Chile Chico),NEGATIVO,NEGATIVO,NaN,...,False,NaN,2021-07-18,Chile Chico,General Carrera,mujer,52,2577.0,2288.0,4865.0
70503,9122491,Coyhaique,2021-03-09,10,DESCARTADA,Búsqueda activa,Actividades gestionadas por la Dirección del S...,NEGATIVO,NaN,NaN,...,False,NaN,2021-03-13,Coyhaique,Coyhaique,hombre,37,28763.0,29055.0,57818.0
90644,11221406,Coyhaique,2021-04-21,16,DESCARTADA,Búsqueda activa,Actividades gestionadas por la Dirección del S...,NEGATIVO,NaN,NaN,...,False,NaN,2021-04-23,Coyhaique,Coyhaique,mujer,26,28763.0,29055.0,57818.0
24783,4354134,Aisén,2020-11-07,45,DESCARTADA,Búsqueda activa,Ejercicio libre de la profesión XI región,NaN,NaN,NaN,...,False,NaN,2020-11-07,Ejercicio Libre de la Profesión,Ejercicio Libre de la Profesión,hombre,35,NaN,NaN,NaN
101560,12147249,Coyhaique,2021-05-12,19,DESCARTADA,Búsqueda activa,Consultorio Víctor Domingo Silva,NaN,NEGATIVO,NaN,...,False,NaN,2021-05-13,Coyhaique,Coyhaique,hombre,43,28763.0,29055.0,57818.0
69280,8992611,Coyhaique,2021-03-06,9,DESCARTADA,Búsqueda activa,Actividades gestionadas por la Dirección del S...,NEGATIVO,NaN,NaN,...,False,NaN,2021-03-09,Coyhaique,Coyhaique,hombre,71,28763.0,29055.0,57818.0
71747,9256039,Aisén,2021-03-12,10,DESCARTADA,Búsqueda activa,Centro de Salud Familiar Puerto Aysen,NEGATIVO,NaN,NaN,...,False,NaN,NaN,Aysen,Aysen,mujer,37,12719.0,11240.0,23959.0
121807,13593953,Coyhaique,2021-06-11,23,DESCARTADA,Búsqueda activa,Actividades gestionadas por la Dirección del S...,NEGATIVO,NaN,NaN,...,False,NaN,2021-06-14,Coyhaique,Coyhaique,mujer,45,28763.0,29055.0,57818.0
16066,3352842,Aisén,2020-10-06,41,DESCARTADA,Búsqueda activa,Ejercicio libre de la profesión XI región,NaN,NaN,NaN,...,False,NaN,2020-10-06,Ejercicio Libre de la Profesión,Ejercicio Libre de la Profesión,hombre,29,NaN,NaN,NaN
75801,9748441,Aisén,2021-03-22,12,DESCARTADA,Búsqueda activa,Centro de Salud Familiar Puerto Aysen,NEGATIVO,NaN,NaN,...,False,NaN,NaN,Aysen,Aysen,hombre,29,12719.0,11240.0,23959.0


In [16]:
df_seguimiento = pd.read_csv('data/df_seguimiento_full.csv',
                            sep=';',
                             low_memory=False
                            )

In [17]:
df_seguimiento

,Unnamed: 0,n_folio,tipo_seguimiento,dia_contacto,id_enfermedad_eno,enfermedad,fecha_notificacion,id_institucion,nombre_institucion_indice,region_institucion_indice,...,cumple_aislamiento_cuarentena,ultimo_establecimiento_derivacion,nombre_ultimo_establecimiento_derivacion,ultimo_seremi_derivacion,región_derivación,región_seremi,región_notificacion,region_final,TTA,file
0,0,8753781,caso,2021-03-02 00:00:00,50,Coronavirus (COVID-19),2021-03-01 00:00:00,125100,Hospital Regional (Coihaique),11,...,t,Sin Datos,Sin Datos,Sin Datos,Sin Datos,Sin Datos,Región De Aysén del General Carlos Ibañez del ...,Región de Aysén del General Carlos Ibáñez del ...,SEREMI,marzo
1,1,SC9677760-11425750,contacto,2021-03-25 00:00:00,50,Coronavirus (COVID-19),2021-03-20 00:00:00,125100,Hospital Regional (Coihaique),11,...,t,Sin Datos,Sin Datos,Sin Datos,Sin Datos,Sin Datos,Región De Aysén del General Carlos Ibañez del ...,Región de Aysén del General Carlos Ibáñez del ...,APS,marzo
2,2,SC9780675-11714657,contacto,2021-03-29 00:00:00,50,Coronavirus (COVID-19),2021-03-23 00:00:00,125010,Actividades gestionadas por la Dirección del S...,11,...,t,Sin Datos,Sin Datos,Sin Datos,Sin Datos,Sin Datos,Región De Aysén del General Carlos Ibañez del ...,Región de Aysén del General Carlos Ibáñez del ...,SEREMI,marzo
3,3,SC10084431-11964234,contacto,2021-03-29 00:00:00,50,Coronavirus (COVID-19),2021-03-29 00:00:00,125010,Actividades gestionadas por la Dirección del S...,11,...,t,Sin Datos,Sin Datos,Sin Datos,Sin Datos,Sin Datos,Región De Aysén del General Carlos Ibañez del ...,Región de Aysén del General Carlos Ibáñez del ...,SEREMI,marzo
4,4,SC10332193-12293320,contacto,2021-04-04 00:00:00,50,Coronavirus (COVID-19),2021-04-03 00:00:00,125404,Posta de Salud Rural Lago Verde,11,...,t,Sin Datos,Sin Datos,Sin Datos,Sin Datos,Sin Datos,Región De Aysén del General Carlos Ibañez del ...,Región de Aysén del General Carlos Ibáñez del ...,SEREMI,marzo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293816,293849,SC16034389-20188901,contacto,2021-08-25 00:00:00,50,Coronavirus (COVID-19),2021-08-15 00:00:00,125100,Hospital Regional (Coihaique),11,...,t,125300,Consultorio Víctor Domingo Silva,Sin Datos,Región De Aysén del General Carlos Ibañez del ...,Sin Datos,Región De Aysén del General Carlos Ibañez del ...,Región de Aysén del General Carlos Ibáñez del ...,APS,agosto
293817,293850,PNTM-7016056768,no notificado índice,Sin Datos,50,Coronavirus (COVID-19),Sin Datos,Sin Datos,Sin Datos,Sin Datos,...,Sin Datos,Sin Datos,Sin Datos,Sin Datos,Sin Datos,Sin Datos,Sin Datos,Región de Aysén del General Carlos Ibáñez del ...,SEREMI,agosto
293818,293851,PNTM-7015472903,no notificado índice,Sin Datos,50,Coronavirus (COVID-19),Sin Datos,Sin Datos,Sin Datos,Sin Datos,...,Sin Datos,Sin Datos,Sin Datos,Sin Datos,Sin Datos,Sin Datos,Sin Datos,Región de Aysén del General Carlos Ibáñez del ...,SEREMI,agosto
293819,293852,PNTM-7015314831,no notificado índice,Sin Datos,50,Coronavirus (COVID-19),Sin Datos,Sin Datos,Sin Datos,Sin Datos,...,Sin Datos,Sin Datos,Sin Datos,Sin Datos,Sin Datos,Sin Datos,Sin Datos,Región de Aysén del General Carlos Ibáñez del ...,SEREMI,agosto


In [18]:
# Divide strings utilizando regex para extraer nfolio de casos, 
# Separa nfolio caso indice de nfolio contacto estrecho y guarda caso índice
# Y extrae datos de PNTM

df_seguimiento['n_folio'].str.extract(r'([0-9]+)|SC[0-9]+-([0-9]+)|PNTM-([0-9]+)', flags=0,expand=True).rename(columns={0:'nfolio_caso',
                                                                                                          1:'nfolio_caso_indice',
                                                                                                          2:'nfolio_pntm'
                                                                                                        })

,nfolio_caso,nfolio_caso_indice,nfolio_pntm
0,8753781,NaN,NaN
1,NaN,11425750,NaN
2,NaN,11714657,NaN
3,NaN,11964234,NaN
4,NaN,12293320,NaN
...,...,...,...
293816,NaN,20188901,NaN
293817,NaN,NaN,7016056768
293818,NaN,NaN,7015472903
293819,NaN,NaN,7015314831


In [19]:
# Separa nfolio caso índice de nfolio contacto estrecho y guarda contacto estrecho

df_seguimiento['n_folio'].str.extract(r'SC([0-9]+)-', 
                                      flags=0,
                                      expand=True).rename(
                                      columns={0:'nfolio_contacto_estrecho'})

,nfolio_contacto_estrecho
0,NaN
1,9677760
2,9780675
3,10084431
4,10332193
...,...
293816,16034389
293817,NaN
293818,NaN
293819,NaN


In [20]:
# Realiza merge con DF original utilizando index de importación
df_seguimiento_merged = df_seguimiento.merge(df_seguimiento['n_folio'].str.extract(
                                                 r'([0-9]+)|SC[0-9]+-([0-9]+)|PNTM-([0-9]+)', 
                                                 flags=0,
                                                 expand=True).rename(
                                                 columns={0:'nfolio_caso',
                                                          1:'nfolio_caso_indice',
                                                          2:'nfolio_pntm'}), 
                                             how='left',
                                             left_index= True,
                                             right_index=True
                                      ).merge(df_seguimiento['n_folio'].str.extract(
                                                                 r'SC([0-9]+)-', 
                                                                 flags=0,
                                                                 expand=True).rename(
                                                                 columns={0:'nfolio_contacto_estrecho'}), 
                                                                 how='left',
                                                                 left_index= True,
                                                                 right_index=True
                                      )

df_seguimiento_merged.sample(3)

,Unnamed: 0,n_folio,tipo_seguimiento,dia_contacto,id_enfermedad_eno,enfermedad,fecha_notificacion,id_institucion,nombre_institucion_indice,region_institucion_indice,...,región_derivación,región_seremi,región_notificacion,region_final,TTA,file,nfolio_caso,nfolio_caso_indice,nfolio_pntm,nfolio_contacto_estrecho
155544,155567,14554539,caso,Sin Datos,50,Coronavirus (COVID-19),2021-07-04 00:00:00,125101,Hospital de Puerto Aysén,11,...,Sin Datos,Sin Datos,Región De Aysén del General Carlos Ibañez del ...,Región de Aysén del General Carlos Ibáñez del ...,APS,junio,14554539,NaN,NaN,NaN
139352,139373,12482259,caso,2021-05-25 00:00:00,50,Coronavirus (COVID-19),2021-05-19 00:00:00,125102,Hospital Dr. Leopoldo Ortega R. (Chile Chico),11,...,Sin Datos,Sin Datos,Región De Aysén del General Carlos Ibañez del ...,Región de Aysén del General Carlos Ibáñez del ...,SEREMI,mayo,12482259,NaN,NaN,NaN
87827,87845,10547247,caso,Sin Datos,50,Coronavirus (COVID-19),2021-04-07 00:00:00,11,Ejercicio libre de la profesión XI región,11,...,Sin Datos,Sin Datos,Sin Datos,Región de Aysén del General Carlos Ibáñez del ...,APS,mayo,10547247,NaN,NaN,NaN


In [21]:
df_seguimiento_merged.head()

,Unnamed: 0,n_folio,tipo_seguimiento,dia_contacto,id_enfermedad_eno,enfermedad,fecha_notificacion,id_institucion,nombre_institucion_indice,region_institucion_indice,...,región_derivación,región_seremi,región_notificacion,region_final,TTA,file,nfolio_caso,nfolio_caso_indice,nfolio_pntm,nfolio_contacto_estrecho
0,0,8753781,caso,2021-03-02 00:00:00,50,Coronavirus (COVID-19),2021-03-01 00:00:00,125100,Hospital Regional (Coihaique),11,...,Sin Datos,Sin Datos,Región De Aysén del General Carlos Ibañez del ...,Región de Aysén del General Carlos Ibáñez del ...,SEREMI,marzo,8753781,NaN,NaN,NaN
1,1,SC9677760-11425750,contacto,2021-03-25 00:00:00,50,Coronavirus (COVID-19),2021-03-20 00:00:00,125100,Hospital Regional (Coihaique),11,...,Sin Datos,Sin Datos,Región De Aysén del General Carlos Ibañez del ...,Región de Aysén del General Carlos Ibáñez del ...,APS,marzo,NaN,11425750,NaN,9677760
2,2,SC9780675-11714657,contacto,2021-03-29 00:00:00,50,Coronavirus (COVID-19),2021-03-23 00:00:00,125010,Actividades gestionadas por la Dirección del S...,11,...,Sin Datos,Sin Datos,Región De Aysén del General Carlos Ibañez del ...,Región de Aysén del General Carlos Ibáñez del ...,SEREMI,marzo,NaN,11714657,NaN,9780675
3,3,SC10084431-11964234,contacto,2021-03-29 00:00:00,50,Coronavirus (COVID-19),2021-03-29 00:00:00,125010,Actividades gestionadas por la Dirección del S...,11,...,Sin Datos,Sin Datos,Región De Aysén del General Carlos Ibañez del ...,Región de Aysén del General Carlos Ibáñez del ...,SEREMI,marzo,NaN,11964234,NaN,10084431
4,4,SC10332193-12293320,contacto,2021-04-04 00:00:00,50,Coronavirus (COVID-19),2021-04-03 00:00:00,125404,Posta de Salud Rural Lago Verde,11,...,Sin Datos,Sin Datos,Región De Aysén del General Carlos Ibañez del ...,Región de Aysén del General Carlos Ibáñez del ...,SEREMI,marzo,NaN,12293320,NaN,10332193


In [22]:
# Revisa folio en DB de notificaciones

data_aysen_clean[data_aysen_clean['numero_folio']==14832232]

,numero_folio,comuna_residencia,fecha_notificacion,semana_epidemiologica,etapa_clinica,tipo_caso_busqueda,establecimiento_salud,resultado_pcr_1,resultado_pcr_2,resultado_pcr_3,...,caso_cerrado,fecha_primeros_sintomas,fecha_diagnostico,comuna_seremi_xi,provincia_seremi_xi,sexo,edad,pop_hombres,pop_mujeres,pop_total
137274,14832232,Coyhaique,2021-07-11,28,CONFIRMADA,Atención médica,Hospital Regional (Coihaique),NEGATIVO,POSITIVO,NaN,...,False,2021-07-09,2021-07-12,Coyhaique,Coyhaique,mujer,47,28763.0,29055.0,57818.0


In [23]:
# Todos los datos poseen un folio 

df_seguimiento_merged[df_seguimiento_merged['n_folio'].isnull()]

,Unnamed: 0,n_folio,tipo_seguimiento,dia_contacto,id_enfermedad_eno,enfermedad,fecha_notificacion,id_institucion,nombre_institucion_indice,region_institucion_indice,...,región_derivación,región_seremi,región_notificacion,region_final,TTA,file,nfolio_caso,nfolio_caso_indice,nfolio_pntm,nfolio_contacto_estrecho


In [24]:
df_seguimiento_merged.columns

Index(['Unnamed: 0', 'n_folio', 'tipo_seguimiento', 'dia_contacto',
       'id_enfermedad_eno', 'enfermedad', 'fecha_notificacion',
       'id_institucion', 'nombre_institucion_indice',
       'region_institucion_indice', 'nombre_region_institucion_indice',
       'institucion_contacto', 'nombre_institucion_seguimiento', 'region',
       'id_region', 'run_profesional', 'nombre_profesional',
       'identificacion_paciente', 'n_identificacion_paciente',
       'nombre_paciente', 'primer_apellido_paciente',
       'fecha_cumple_requisitos', 'cont_tipo_identificacion',
       'cont_n_identificacion', 'cont_nombres', 'cont_primer_apellido',
       'cont_segundo_apellido', 'cont_sexo', 'cont_fecha_nacimiento',
       'cont_edad', 'cont_tipo_direccion', 'cont_tipo_institucion',
       'cont_via_residencia', 'cont_direccions', 'cont_n_residencia',
       'cont_dpto', 'cont_poblacion', 'cont_region', 'cont_comuna',
       'cont_n_telefono', 'cont_n_celular', 'cont_correo_electronico',
       '

In [25]:
pd.DataFrame(df_seguimiento_merged.columns).rename(columns={0:'variables_seguimiento'}).transpose()

,0,1,2,3,4,5,6,7,8,9,...,80,81,82,83,84,85,86,87,88,89
variables_seguimiento,Unnamed: 0,n_folio,tipo_seguimiento,dia_contacto,id_enfermedad_eno,enfermedad,fecha_notificacion,id_institucion,nombre_institucion_indice,region_institucion_indice,...,región_derivación,región_seremi,región_notificacion,region_final,TTA,file,nfolio_caso,nfolio_caso_indice,nfolio_pntm,nfolio_contacto_estrecho


In [26]:
df_seguimiento_merged['folio_se_convierte_en_caso']

0         Sin Datos
1         Sin Datos
2         Sin Datos
3         Sin Datos
4         Sin Datos
            ...    
293816    Sin Datos
293817    Sin Datos
293818    Sin Datos
293819    Sin Datos
293820     16161119
Name: folio_se_convierte_en_caso, Length: 293821, dtype: object

In [27]:
# Revisa existencia de una variable dentro del DF y sus respectivos valores

df_seguimiento_merged['folio_se_convierte_en_caso'].value_counts()


Sin Datos    277894
-1              528
14560708         36
10748118         33
10746410         33
              ...  
16161119          1
12177212          1
13618714          1
13140668          1
12127513          1
Name: folio_se_convierte_en_caso, Length: 2669, dtype: int64

# Variables de interes


|Variable|Tipo|Observación|
|---|---|---|
|`n_folio`|Text|folio generado por Número y Letras|
|`nfolio_caso`|Numeric|folio de caso registrado|
|`nfolio_caso_indice`|Numeric|folio de caso indice registrado|
|`nfolio_pntm`|Numeric|folio plataforma nacional de toma de muestra|
|`nfolio_contacto_estrecho`|Numeric|folio de contacto estrecho|
|`tipo_seguimiento`|Text|contacto, caso, no notificado indice|
|`dia_contacto`|DateTime|Fecha de contacto |
|`fecha_notificacion`|DateTime|Fecha de notificación|
|`nombre_institucion_indice`|Text|Nombre institución desde donde se deriva el caso|
|`region_institucion_indice`|Numeric| Región d ela cual se deriva el caso|
|`nombre_region_institucion_indice`|Text|Nombre de región de la institución índice|
|`nombre_institucion_seguimiento`|Text|Nombre de la institución encargada de realizar seguimiento|
|`region`|Text|Región en la cual se realiza el seguimiento|
|`id_region`|Numeric|Número de la región de seguimiento|
|`fecha_cumple_requisitos`|DateTime|Fecha en la cual cumple requisitos para seguimiento|
|`TTA`|Text|APS o SEREMI|
|`cumple_aislamiento_cuarentena`|Boolean|Sin datos, t, f|
|`folio_se_convierte_en_caso`|Text|Número de folio o Sin Datos|
|`visible_no_eliminado`|Boolean|Sin datos, t, f|
|`suspendido`|Text|egresado|
|`sintomas`|Text|Sintomatología, Sin datos o Ninguno|
|`observacion_seguimiento_contactos`|Text| Escrito por el trazador|
|`tiene_resultado_covid`|Text|Positivo, Negativo, No concluyente, Sin Datos|
|`derivacion_a_su`|Boolean| t f Sin datos|
|`derivacion_toma_muestra`|Boolean|t f Sin datos|
|`fecha_derivacion_toma_muestra`|Text|Fecha Sin datos|
|`cont_fecha_seguimiento`|Text|Texto o Sin Datos|
|`cont_tipo_contacto`|Text|contacto_familiar, contacto_social, contacto_institucional, contacto_laboral|
|||contacto_pasajero, contacto_sala_espera, contacto_personal_salud,Datos|
|`cont_parentesco`|Text|Hijo/a, Madre/Padre, Otro familiar, Pareja, Hermano/a, Otra relación/No familiar, Sin Datos|
|`cont_otra_relacion`|Text|Sin Datos, otro, hijo de cuidadora, otro adulto mayor en cuidado, cuñado. sobrino de cuidadora, AMIGO,  Amigo|
||| pareja del tío de su madre,cuidadora, PAREJA PADRE, otro adulto en cuidados, Pareja de mamá, Hijastro|
|`cont_tipo_transporte`|Text|Aéreo, Sin Datos, Terrestre|
|`cont_nombre_vuelo`|Text|Sin Datos, JETSMART JA245 CCP - BALMACEDA, SKY 433, COPA CM111, 
|||SKY 433 STGO- COYHAIQUE 21-08-2021 / 11 AM, SK, DL147 ATLANTA - STGO, JETSMART CCP-BALMACEDA, LA 272 |
|||JETSMART JA245, VUELO CM111, LA272, JETSMART JA 245 CCP - BALMACEDA|                 
|`cont_fecha_llegada`|Text|Sin Datos, 2021-07-21, 2021-08-21, 2021-07-01, 2021-07-03, 2021-07-09, 2021-07-02|
|`detalle_pasajero`|Text|Observaciones adicionales de viaje|
|`cont_inicio_cuarentena`|DateTime|Fecha de inicio de cuarentena|
|`cont_fin_cuarentena`|DateTime|Fecha y Sin Datos|
|`cont_requiere_licencia_medica`|Boolean|Sin Datos, NO, SI, Seleccione|
|`cont_se_emite_licencia`|Boolean|Sin Datos, t, f|
|`cont_fecha_licencia`|DateTime|Fecha de licencia|
|`requiere_canasta_alimentos`|Boolean|Sin Datos, t, f|
|`cantidad_personas_vivienda`|Numeric|Integrantes familiares de 0 a 6|
|`cont_estado_seguimiento`|Text|Aún no contactado, En proceso de seguimiento, Se transforma en caso, Contacto sintomático, No localizado|
|`fecha_creacion`|DateTime|Fecha de creación|
|`tipo_del_contacto`|Text|Sin Datos, Llamada telefónica, Visita domiciliaria|
|`contacto_localizado`|Boolean|Sin Datos, t, f|
|`empresa`|Text|Información de Empresa de trabajo|
|`institucion`|Text|Información de la institución|

## Limpieza y ordenamiento de datos



Segun variables seleccionadas como de importancia de análisis y obsrvaciones a filtrar

In [28]:
# Transforma fecha_cumple_requisitos a formato DateTime64 

df_seguimiento_merged['fecha_cumple_requisitos'] = pd.to_datetime(df_seguimiento_merged['fecha_cumple_requisitos'], errors='coerce')
df_seguimiento_merged['fecha_notificacion'] = pd.to_datetime(df_seguimiento_merged['fecha_notificacion'], errors='coerce')
df_seguimiento_merged['fecha_creacion'] = pd.to_datetime(df_seguimiento_merged['fecha_creacion'], errors='coerce')

#Crea diferencia de fecha
df_seguimiento_merged['dif_fecha'] = df_seguimiento_merged['fecha_cumple_requisitos'] - df_seguimiento_merged['fecha_notificacion']

In [29]:
# Filtra los timedelta mayores a 14 dias

seg = df_seguimiento_merged[df_seguimiento_merged['dif_fecha']<pd.to_timedelta('14 days')]

In [30]:
#Filtra visible_no_eliminado y bota duplicados en n_folio

aux = seg[seg['visible_no_eliminado']=='t'].drop_duplicates('n_folio')

In [31]:
# Filtra tipo_seguimiento para los casos 

aux_caso = aux[aux['tipo_seguimiento']=='caso']

In [32]:
aux_contacto = aux[aux['tipo_seguimiento']=='contacto']
#Crea columna dif_fecha_contacto
aux_contacto['dif_fecha_contacto'] = aux_contacto['fecha_creacion']-aux_contacto['fecha_notificacion']

aux_contacto = aux_contacto[aux_contacto['dif_fecha_contacto']>=pd.to_timedelta(-14)]

<ipython-input-32-483468821897>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aux_contacto['dif_fecha_contacto'] = aux_contacto['fecha_creacion']-aux_contacto['fecha_notificacion']


In [33]:
aux_contacto_nuevos = aux[aux['tipo_seguimiento']=='contacto']
aux_contacto_nuevos = aux_contacto_nuevos[(aux_contacto_nuevos['suspendido']=='activo')|(aux_contacto_nuevos['suspendido']=='egresado')]


In [34]:
"""aux_contacto_viejos = aux_contacto[(aux_contacto['fecha_creacion']-pd.to_timedelta('7 days'))]

aux_contacto_viejos= aux_contacto_viejos[(aux_contacto_viejos['suspendido']=='activo')|(aux_contacto_viejos['suspendido']=='egresado')]"""


"aux_contacto_viejos = aux_contacto[(aux_contacto['fecha_creacion']-pd.to_timedelta('7 days'))]\n\naux_contacto_viejos= aux_contacto_viejos[(aux_contacto_viejos['suspendido']=='activo')|(aux_contacto_viejos['suspendido']=='egresado')]"

In [35]:
#aux_contacto[
(aux_contacto['fecha_creacion']-pd.to_timedelta('7 days'))
#]

1        2021-03-15 12:49:44.051748
2        2021-03-18 18:23:53.976693
3        2021-03-22 17:55:07.453245
4        2021-03-27 13:40:31.874120
5        2021-03-29 20:26:57.278563
                    ...            
293456   2021-08-13 19:40:56.582091
293485   2021-08-05 18:15:06.027185
293525   2021-08-09 23:23:22.867003
293560   2021-08-21 11:34:20.083839
293814   2021-08-10 19:43:02.977625
Name: fecha_creacion, Length: 15525, dtype: datetime64[ns]

In [36]:
aux_contacto['fecha_creacion']-pd.to_timedelta(-7)

1        2021-03-22 12:49:44.051748007
2        2021-03-25 18:23:53.976693007
3        2021-03-29 17:55:07.453245007
4        2021-04-03 13:40:31.874120007
5        2021-04-05 20:26:57.278563007
                      ...             
293456   2021-08-20 19:40:56.582091007
293485   2021-08-12 18:15:06.027185007
293525   2021-08-16 23:23:22.867003007
293560   2021-08-28 11:34:20.083839007
293814   2021-08-17 19:43:02.977625007
Name: fecha_creacion, Length: 15525, dtype: datetime64[ns]

In [37]:
aux_contacto['fecha_creacion']

1        2021-03-22 12:49:44.051748
2        2021-03-25 18:23:53.976693
3        2021-03-29 17:55:07.453245
4        2021-04-03 13:40:31.874120
5        2021-04-05 20:26:57.278563
                    ...            
293456   2021-08-20 19:40:56.582091
293485   2021-08-12 18:15:06.027185
293525   2021-08-16 23:23:22.867003
293560   2021-08-28 11:34:20.083839
293814   2021-08-17 19:43:02.977625
Name: fecha_creacion, Length: 15525, dtype: datetime64[ns]

In [38]:
df_seguimiento_merged['visible_no_eliminado']

0         t
1         t
2         t
3         t
4         t
         ..
293816    t
293817    t
293818    t
293819    t
293820    t
Name: visible_no_eliminado, Length: 293821, dtype: object

In [39]:
data_aysen_clean

,numero_folio,comuna_residencia,fecha_notificacion,semana_epidemiologica,etapa_clinica,tipo_caso_busqueda,establecimiento_salud,resultado_pcr_1,resultado_pcr_2,resultado_pcr_3,...,caso_cerrado,fecha_primeros_sintomas,fecha_diagnostico,comuna_seremi_xi,provincia_seremi_xi,sexo,edad,pop_hombres,pop_mujeres,pop_total
0,50166,Coyhaique,2020-03-12,11,DESCARTADA,NaN,Hospital Regional (Coihaique),NEGATIVO,NaN,NaN,...,False,2020-03-10,2020-03-16,Coyhaique,Coyhaique,mujer,46,28763.0,29055.0,57818.0
2,50785,Coyhaique,2020-03-12,11,DESCARTADA,NaN,Hospital Regional (Coihaique),NEGATIVO,NaN,NaN,...,False,2020-03-10,2020-03-28,Coyhaique,Coyhaique,hombre,12,28763.0,29055.0,57818.0
3,50794,Coyhaique,2020-03-12,11,DESCARTADA,NaN,Hospital Regional (Coihaique),NEGATIVO,NaN,NaN,...,False,2020-03-10,2020-03-28,Coyhaique,Coyhaique,hombre,11,28763.0,29055.0,57818.0
5,53499,Cisnes,2020-03-18,12,DESCARTADA,NaN,Centro de Salud Familiar La Junta,NEGATIVO,NaN,NaN,...,False,2020-03-15,2020-03-24,Cisnes,Aysen,mujer,48,3922.0,2595.0,6517.0
7,54731,Cisnes,2020-03-19,12,DESCARTADA,Atención médica,Centro de Salud Familiar La Junta,NO CONCLUYENTE,NaN,NaN,...,False,2020-03-19,2020-03-21,Cisnes,Aysen,hombre,0,3922.0,2595.0,6517.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161660,16681784,Cochrane,2021-09-04,36,SOSPECHA,Atención médica,Hospital Lord Cochrane,NEGATIVO,NaN,NaN,...,False,2021-09-03,NaN,Cochrane,Capitan Prat,mujer,37,1902.0,1588.0,3490.0
161661,16681803,Cochrane,2021-09-04,36,SOSPECHA,Atención médica,Hospital Lord Cochrane,NaN,NaN,NaN,...,False,2021-09-04,NaN,Cochrane,Capitan Prat,hombre,6,1902.0,1588.0,3490.0
161662,16681835,Cochrane,2021-09-04,36,SOSPECHA,Atención médica,Hospital Lord Cochrane,NaN,NaN,NaN,...,False,2021-09-03,NaN,Cochrane,Capitan Prat,hombre,11,1902.0,1588.0,3490.0
161663,16682064,Aisén,2021-09-05,36,SOSPECHA,Atención médica,Hospital de Puerto Aysén,NaN,NaN,NaN,...,False,2021-08-31,NaN,Aysen,Aysen,mujer,5,12719.0,11240.0,23959.0


In [46]:
df_seguimiento_filtered = df_seguimiento_merged

In [47]:
# Revisa tipo de datos
df_seguimiento_filtered['nfolio_caso'].dtype

dtype('O')

In [103]:
# Revisa tipo de datos para merging

df_seguimiento_filtered['numero_folio'].dtype

dtype('float64')

In [49]:
# Transforma dtype de ambas columnas a float64 para permitir merging 

df_seguimiento_filtered['nfolio_caso']= df_seguimiento_filtered['nfolio_caso'].astype('float64', errors='raise')


data_aysen_clean['numero_folio']= data_aysen_clean['numero_folio'].astype('float64', errors='raise')

<ipython-input-49-e6def0a43977>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_aysen_clean['numero_folio']= data_aysen_clean['numero_folio'].astype('float64', errors='raise')


In [50]:
# Combina ambas columnas para nfolio_caso y nfolio_contacto_estrecho y 
# agrega NA en donde corresponda a otros casos como PNTM o nfolio_caso_indice

df_seguimiento_filtered['nfolio_merge'] = df_seguimiento_filtered['nfolio_caso'].combine_first(df_seguimiento_filtered['nfolio_contacto_estrecho']).astype('float64', errors='raise')



In [51]:
# Realiza merge con base de datos de notificaciones agregando los valores de casos confirmados y probables


df_seguimiento_filtered= df_seguimiento_filtered.merge(data_aysen_clean[[
                                            'numero_folio',
                                            'etapa_clinica',
                                            'tipo_caso_busqueda',
                                            'establecimiento_salud',
                                            'resultado_pcr_1',
                                            'resultado_pcr_2',
                                            'resultado_pcr_3',
                                            'fecha_resultado_pcr_1',
                                            'fecha_resultado_pcr_2',
                                            'fecha_resultado_pcr_3',
                                            'comuna_seremi_xi',
                                            'provincia_seremi_xi',
                                            'edad',
                                            'sexo',
                                            'pop_hombres',
                                            'pop_mujeres',
                                            'pop_total'
                                        ]], left_on='nfolio_merge', right_on='numero_folio' , how='left')

In [118]:
# Selecciona variables de importancia

df_seguimiento_clean = df_seguimiento_filtered[[
    'n_folio',
    'nfolio_caso',
    'nfolio_caso_indice',
    'nfolio_pntm',
    'nfolio_contacto_estrecho',
    'tipo_seguimiento',
    'dia_contacto',
    'fecha_notificacion',
    'nombre_institucion_indice',
    'region_institucion_indice',
    'nombre_region_institucion_indice',
    'region',
    'id_region',
    'fecha_cumple_requisitos',
    'TTA',
    'cumple_aislamiento_cuarentena',
    'folio_se_convierte_en_caso',
    'visible_no_eliminado',
    'suspendido',
    'sintomas',
    'observacion_seguimiento_contactos',
    'tiene_resultado_covid',
    'derivacion_a_su',
    'derivacion_toma_muestra',
    'fecha_derivacion_toma_muestra',
    'cont_fecha_seguimiento',
    'cont_tipo_contacto',
    'cont_parentesco',
    'cont_otra_relacion',
    'cont_tipo_transporte',
    'cont_nombre_vuelo',
    'cont_fecha_llegada',
    'detalle_pasajero',
    'cont_inicio_cuarentena',
    'cont_fin_cuarentena',
    'cont_requiere_licencia_medica',
    'cont_se_emite_licencia',
    'cont_fecha_licencia',
    'requiere_canasta_alimentos',
    'cantidad_personas_vivienda',
    'cont_estado_seguimiento',
    'fecha_creacion',
    'tipo_del_contacto',
    'contacto_localizado',
    'empresa',
    'institucion',
    'etapa_clinica',
    'tipo_caso_busqueda',
    'establecimiento_salud',
    'resultado_pcr_1',
    'resultado_pcr_2',
    'resultado_pcr_3',
    'fecha_resultado_pcr_1',
    'fecha_resultado_pcr_2',
    'fecha_resultado_pcr_3',
    'comuna_seremi_xi',
    'provincia_seremi_xi',
    'edad',
    'sexo',
    'pop_hombres',
    'pop_mujeres',
    'pop_total'
]]

In [119]:
df_seguimiento_clean

,n_folio,nfolio_caso,nfolio_caso_indice,nfolio_pntm,nfolio_contacto_estrecho,tipo_seguimiento,dia_contacto,fecha_notificacion,nombre_institucion_indice,region_institucion_indice,...,fecha_resultado_pcr_1,fecha_resultado_pcr_2,fecha_resultado_pcr_3,comuna_seremi_xi,provincia_seremi_xi,edad,sexo,pop_hombres,pop_mujeres,pop_total
0,8753781,8753781.0,NaN,NaN,NaN,caso,2021-03-02 00:00:00,2021-03-01,Hospital Regional (Coihaique),11,...,2021-03-02,2021-03-26,NaT,Coyhaique,Coyhaique,47.0,hombre,28763.0,29055.0,57818.0
1,SC9677760-11425750,NaN,11425750,NaN,9677760,contacto,2021-03-25 00:00:00,2021-03-20,Hospital Regional (Coihaique),11,...,2021-03-22,2021-03-21,NaT,Coyhaique,Coyhaique,19.0,mujer,28763.0,29055.0,57818.0
2,SC9780675-11714657,NaN,11714657,NaN,9780675,contacto,2021-03-29 00:00:00,2021-03-23,Actividades gestionadas por la Dirección del S...,11,...,NaT,NaT,NaT,Coyhaique,Coyhaique,31.0,mujer,28763.0,29055.0,57818.0
3,SC10084431-11964234,NaN,11964234,NaN,10084431,contacto,2021-03-29 00:00:00,2021-03-29,Actividades gestionadas por la Dirección del S...,11,...,2021-03-31,NaT,NaT,Coyhaique,Coyhaique,15.0,mujer,28763.0,29055.0,57818.0
4,SC10332193-12293320,NaN,12293320,NaN,10332193,contacto,2021-04-04 00:00:00,2021-04-03,Posta de Salud Rural Lago Verde,11,...,NaT,NaT,NaT,Lago Verde,Coyhaique,82.0,hombre,494.0,358.0,852.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293816,SC16034389-20188901,NaN,20188901,NaN,16034389,contacto,2021-08-25 00:00:00,2021-08-15,Hospital Regional (Coihaique),11,...,2021-08-16,2021-08-19,NaT,Coyhaique,Coyhaique,5.0,mujer,28763.0,29055.0,57818.0
293817,PNTM-7016056768,NaN,NaN,7016056768,NaN,no notificado índice,Sin Datos,NaT,Sin Datos,Sin Datos,...,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
293818,PNTM-7015472903,NaN,NaN,7015472903,NaN,no notificado índice,Sin Datos,NaT,Sin Datos,Sin Datos,...,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
293819,PNTM-7015314831,NaN,NaN,7015314831,NaN,no notificado índice,Sin Datos,NaT,Sin Datos,Sin Datos,...,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [120]:
df_seguimiento_clean['nfolio_caso_indice'].value_counts()

12436348    39
13041090    33
12763425    33
12764155    33
12834928    33
            ..
20120942     1
14355361     1
20097646     1
20606683     1
14543039     1
Name: nfolio_caso_indice, Length: 15881, dtype: int64

In [154]:
#Se consideran solo los visible no eliniminados y se botan duplicados

df_seguimiento_clean= df_seguimiento_clean[df_seguimiento_clean['visible_no_eliminado']=='t'].drop_duplicates()


In [156]:
df_seguimiento_clean['nfolio_caso_indice'].value_counts()

13041090    22
15506064    22
19425664    22
15573170    21
15572891    21
            ..
16685014     1
18588652     1
11889988     1
20202330     1
18492601     1
Name: nfolio_caso_indice, Length: 15717, dtype: int64

# Indicador 5: Oportunidad en la investigación epidemiológica de casos

## Objetivo: Fortalecer la investigación epidemiológica de los casos

### Meta: >90%

**Periodicidad:** Semanal

**Escala Territorial:** Comunal, Regional, Nacional

**Fuente de Información:** Base EPIVIGILIA actualizada con los reportes de laboratorio al día de la determinación del indicador. Base de seguimiento

### Cálculo del indicador a nivel comunal

$\frac{No.\:casos\:nuevos\:registrados\:investigados\:dentro\:de\:48\:hrs\:por\:comuna\:1er\:establecimiento,\:semana\:x}{No.\:casos\:nuevos\:registrados\:por\:comuna\:1er\:establecimiento\:semana\:x}x100$

### Cálculo del indicador a nivel regional

$\frac{No.\:casos\:nuevos\:registrados\:investigados\:dentro\:de\:48\:hrs\:por\:región\:1er\:establecimiento,\:semana\:x}{No.\:casos\:nuevos\:registrados\:por\:región\:1er\:establecimiento\:semana\:x}x100$

**Observación**

- No se incluyen los casos nuevos no notificados
- Se considera un caso nuevo registrado a aquel caso notificado que cumple los requisitos para estar en el modulo de seguimiento
- La fecha para determinar el *inicio de seguimiento* corresponde a la **fecha de resultado (caso confirmado)** o **fecha de notificación (caso probable)**. Para un caso confirmado, si el resultado del test es anterior a la fecha de notificación, se debe iniciar seguimiento a partir de la fecha en que el registro está disponible en la bandeja de seguimiento.
- Se cumple el indicador si, dentro de las 48 hrs. a partir de la fecha de inicio de seguimiento se tiene al menos una actividad de seguimiento exitosa (se logra ubicar al caso).
- El primer establecimiento corresponde al establecimiento dado por API FONASA. Si el registro no hace match con API FONASA, se considera el establecimiento de notificación.
- Se consideran sólo registros de casos que estén activos o egresados en la bandeja de seguimiento.

In [157]:
df_seguimiento_clean['fecha_creacion'] #casos nuevos registrados

0        2021-03-01 11:46:59.255357
1        2021-03-22 12:49:44.051748
2        2021-03-25 18:23:53.976693
3        2021-03-29 17:55:07.453245
4        2021-04-03 13:40:31.874120
                    ...            
293816   2021-08-17 12:10:55.321584
293817   2021-08-16 00:32:44.242543
293818   2021-08-05 00:32:25.801614
293819   2021-08-01 00:32:32.553682
293820   2021-08-19 00:33:56.129119
Name: fecha_creacion, Length: 177558, dtype: datetime64[ns]

In [158]:

#Se consideran sólo registros de casos que estén activos o egresados en la bandeja de seguimiento.

df_seguimiento_ind5= df_seguimiento_clean[(df_seguimiento_clean['suspendido']=='activo')|(df_seguimiento_clean['suspendido']=='egresado')]

df_seguimiento_ind5.sample(2)

,n_folio,nfolio_caso,nfolio_caso_indice,nfolio_pntm,nfolio_contacto_estrecho,tipo_seguimiento,dia_contacto,fecha_notificacion,nombre_institucion_indice,region_institucion_indice,...,fecha_resultado_pcr_1,fecha_resultado_pcr_2,fecha_resultado_pcr_3,comuna_seremi_xi,provincia_seremi_xi,edad,sexo,pop_hombres,pop_mujeres,pop_total
144817,SC12367433-15219853,NaN,15219853,NaN,12367433,contacto,2021-05-18 00:00:00,2021-05-17,SAPU Dr. Alejandro Gutiérrez,11,...,2021-05-18,NaT,NaT,Coyhaique,Coyhaique,14.0,hombre,28763.0,29055.0,57818.0
204559,13827980,13827980.0,NaN,NaN,NaN,caso,2021-06-26 00:00:00,2021-06-16,Actividades gestionadas por la Dirección del S...,11,...,NaT,2021-06-17,NaT,Coyhaique,Coyhaique,43.0,mujer,28763.0,29055.0,57818.0


In [159]:
df_seguimiento_ind5[['suspendido', 'etapa_clinica', 'visible_no_eliminado','resultado_pcr_1','resultado_pcr_2','resultado_pcr_3']]

,suspendido,etapa_clinica,visible_no_eliminado,resultado_pcr_1,resultado_pcr_2,resultado_pcr_3
0,egresado,CONFIRMADA,t,POSITIVO,POSITIVO,NaN
1,egresado,CONFIRMADA,t,POSITIVO,POSITIVO,NaN
2,egresado,CONFIRMADA,t,NaN,NaN,NaN
3,egresado,CONFIRMADA,t,POSITIVO,NaN,NaN
4,egresado,PROBABLE,t,NaN,NaN,NaN
...,...,...,...,...,...,...
293816,egresado,CONFIRMADA,t,POSITIVO,POSITIVO,NaN
293817,egresado,NaN,t,NaN,NaN,NaN
293818,egresado,NaN,t,NaN,NaN,NaN
293819,egresado,NaN,t,NaN,NaN,NaN


In [160]:
# Aún existen casos con 'etapa_clinica' DESCARTADA y SOSPECHA 

df_seguimiento_ind5.etapa_clinica.value_counts()

CONFIRMADA    82340
PROBABLE       9364
DESCARTADA      475
SOSPECHA         43
Name: etapa_clinica, dtype: int64

In [161]:

df_seguimiento_ind5[df_seguimiento_ind5['etapa_clinica']=='CONFIRMADA'][
    ['etapa_clinica',
     'tipo_seguimiento',
     'fecha_cumple_requisitos', 
     'fecha_notificacion',
     'fecha_resultado_pcr_1',
     'fecha_resultado_pcr_2',
     'fecha_resultado_pcr_3']
]

,etapa_clinica,tipo_seguimiento,fecha_cumple_requisitos,fecha_notificacion,fecha_resultado_pcr_1,fecha_resultado_pcr_2,fecha_resultado_pcr_3
0,CONFIRMADA,caso,2021-03-01,2021-03-01,2021-03-02,2021-03-26,NaT
1,CONFIRMADA,contacto,2021-03-22,2021-03-20,2021-03-22,2021-03-21,NaT
2,CONFIRMADA,contacto,2021-03-25,2021-03-23,NaT,NaT,NaT
3,CONFIRMADA,contacto,2021-03-29,2021-03-29,2021-03-31,NaT,NaT
5,CONFIRMADA,contacto,2021-04-05,2021-04-05,NaT,NaT,NaT
...,...,...,...,...,...,...,...
293805,CONFIRMADA,contacto,2021-08-14,2021-08-12,2021-08-14,2021-06-05,NaT
293806,CONFIRMADA,contacto,2021-08-17,2021-08-15,2021-08-16,2021-08-19,NaT
293807,CONFIRMADA,contacto,2021-08-19,2021-08-17,NaT,NaT,NaT
293815,CONFIRMADA,contacto,2021-08-18,2021-08-16,2021-08-17,2021-08-19,NaT


In [480]:
# Elimina etapa_clinica DESCARTADA y SOSPECHA
df_seguimiento_ind5 = df_seguimiento_ind5[(df_seguimiento_ind5['etapa_clinica']!='DESCARTADA')|(df_seguimiento_ind5['etapa_clinica']!='SOSPECHA')]

In [505]:
# Toma en cuenta solo el primer contacto con el contacto o caso

df_seguimiento_ind5= df_seguimiento_ind5.sort_values(by=['n_folio', 'dia_contacto']).groupby('n_folio').head(1)

In [506]:
#Crea diferencia de fecha
df_seguimiento_ind5['dif_fecha'] = df_seguimiento_ind5['fecha_cumple_requisitos'] - df_seguimiento_ind5['fecha_notificacion']

In [507]:
# Crea columna considerando si diferencial de fecha cumple requisitos y fecha de notificacion
# es menor a un timedelta de d 2 días

df_seguimiento_ind5['ind5'] = df_seguimiento_ind5['dif_fecha']<= pd.to_timedelta('2 days')

In [508]:
df_seguimiento_ind5[['fecha_creacion',
                       'fecha_notificacion', 
                       'fecha_cumple_requisitos', #Se considera un caso nuevo registrado a aquel caso notificado que cumple los requisitos para estar en el modulo de seguimiento
                       'fecha_derivacion_toma_muestra',
                       'cont_fecha_seguimiento',
                       'cont_inicio_cuarentena',
                       'cont_fin_cuarentena',
                       'cont_fecha_licencia',
                       'dif_fecha'
                      ]].sample(4)

,fecha_creacion,fecha_notificacion,fecha_cumple_requisitos,fecha_derivacion_toma_muestra,cont_fecha_seguimiento,cont_inicio_cuarentena,cont_fin_cuarentena,cont_fecha_licencia,dif_fecha
128289,2021-05-19 21:03:30.902947,2021-05-15,2021-05-19,Sin Datos,Sin Datos,2021-05-15 00:00:00,2021-05-28 00:00:00,Sin Datos,4 days
219737,2021-06-24 00:34:24.191499,NaT,2021-06-23,Sin Datos,Sin Datos,2021-06-23 00:00:00,2021-07-03 00:00:00,Sin Datos,NaT
211205,2021-06-07 18:02:24.857389,2021-06-07,2021-06-07,2021-06-07 00:00:00,Sin Datos,2021-06-07 00:00:00,2021-06-20 00:00:00,Sin Datos,0 days
275924,2021-08-23 09:56:09.309675,2021-08-23,2021-08-23,Sin Datos,Sin Datos,2021-08-16 00:00:00,2021-08-26 00:00:00,Sin Datos,0 days


In [509]:
df_seguimiento_ind5.groupby(['fecha_cumple_requisitos', 'ind5']).count()[['n_folio']].reset_index()

,fecha_cumple_requisitos,ind5,n_folio
0,2021-03-01,True,27
1,2021-03-02,True,62
2,2021-03-03,True,56
3,2021-03-04,False,6
4,2021-03-04,True,33
...,...,...,...
339,2021-08-27,True,13
340,2021-08-28,True,6
341,2021-08-29,True,20
342,2021-08-30,True,82


In [510]:
# Crea gráfico de etapa_clinica regional por semana epidemiologica

px.bar(df_seguimiento_ind5.groupby(['fecha_cumple_requisitos', 'ind5']).count()[['n_folio']].reset_index(),
        x= 'fecha_cumple_requisitos',
        y= 'n_folio',
        color= 'ind5',
        labels={'ind5':'Cumple'}, 
       ).update_layout(title= 'Casos de la Región de Aysén diferenciados por etapa clínica',
                    yaxis_title='Número de casos',
                    xaxis_title='Semana epidemiológica',
                    template='simple_white',    )                 
                    #xaxis=dict(tickmode = 'array',
                     #             tickvals = list(range(12,88,3)),
                      #            ticktext = list(list(range(12,54,3))+list(range(1,88,3))),
                      #           ),
               #     ).add_vline(x=53.5, line_width=3, line_dash='dash', line_color='gray'
                #               ).add_vrect(x0='50', x1='50', 
                 #                          annotation_text='2020', 
                  #                         annotation_position='top right',
                   #                        annotation=dict(font_size=18, 
                    #                                       font_family="'Open Sans', verdana, arial, sans-serif"),
                     #                                      opacity=0, 
                      #                                     line_width=0
                       #                   ).add_vrect(x0='84', x1='86', 
                        #                   annotation_text='2021', 
                         #                  annotation_position='top right',
                          #                 annotation=dict(font_size=18, 
                           #                                font_family="'Open Sans', verdana, arial, sans-serif"),
                            #                               opacity=0, 
                             #                              line_width=0)  
                      

In [511]:
df_ind5= df_seguimiento_ind5.groupby(['fecha_cumple_requisitos', 'ind5']).count()[['n_folio']].reset_index()

In [512]:
df_ind5_perc = (df_ind5[df_ind5['ind5']==True].set_index('fecha_cumple_requisitos')[['n_folio']]/df_ind5.groupby('fecha_cumple_requisitos').sum('n_folio')[['n_folio']])*100

In [513]:
px.scatter(df_ind5_perc.reset_index(),
        x= 'fecha_cumple_requisitos',
        y= 'n_folio',
        color='n_folio',
        color_continuous_scale=px.colors.sequential.Viridis_r,
        labels={'n_folio':'Oportunidad (%)'},
        ).update_layout(title= 'Oportunidad en la investigación epidemiológica de casos',
                       yaxis_title='Oportunidad (%)',
                       xaxis_title='Fecha',
                       template='simple_white',
                 #      xaxis=dict(tickmode = 'array',
                  #                tickvals = list(range(12,88,3)),
                   #               ticktext = list(list(range(12,54,3))+list(range(1,88,3))),
                    #             ),
                 #   ).add_vline(x=53.5, line_width=3, line_dash='dash', line_color='gray'
                  #             ).add_hline(y=10, 
                   #                        line_width=3, 
                    #                       line_dash='longdash', 
                     #                      line_color='olive',
                      #                     annotation_text='Meta nacional (10%)',
                       #                    annotation_position='bottom right'
                        #         ).add_hline(y=mean_bac_ind, 
                         #                    line_width=3, 
                          #                   line_dash='longdash', 
                           #                  line_color='green',
                            #                 annotation_text= 'Promedio BAC ('+str(mean_bac_ind)+'%)',
                             #                annotation_position='bottom right'
                              #       ).add_vrect(x0='51', x1='50', 
                               #            annotation_text='2020', 
                                #           annotation_position='top right',
                                 #          annotation=dict(font_size=18, 
                                  #                         font_family="'Open Sans', verdana, arial, sans-serif"),
                                   #                        opacity=0, 
                                    #                       line_width=0
                                     #     ).add_vrect(x0='84', x1='86', 
                                      #     annotation_text='2021', 
                                       #    annotation_position='top right',
                                        #   annotation=dict(font_size=18, 
                                         #                  font_family="'Open Sans', verdana, arial, sans-serif"),
                                          #                 opacity=0, 
                                           #                line_width=0
                                                     ).update_traces(mode='lines+markers', 
                                                                     line_color='grey',
                                                                     marker_line_width=1, 
                                                                     marker_size=10)

In [514]:
df_ind5_perc.reset_index().head()

,fecha_cumple_requisitos,n_folio
0,2021-03-01,100.000000
1,2021-03-02,100.000000
2,2021-03-03,100.000000
3,2021-03-04,84.615385
4,2021-03-05,83.050847


In [515]:
data_aysen_clean_21[['fecha_notificacion','semana_epidemiologica_21']]

,fecha_notificacion,semana_epidemiologica_21
0,2020-03-12,11
1,2020-03-12,11
2,2020-03-12,11
3,2020-03-18,12
4,2020-03-19,12
...,...,...
151482,2021-09-04,89
151483,2021-09-04,89
151484,2021-09-04,89
151485,2021-09-05,89


In [516]:
df_ind5_perc.reset_index().merge(data_aysen_clean_21[['fecha_notificacion','semana_epidemiologica_21']], 
                   left_on='fecha_cumple_requisitos',
                   right_on='fecha_notificacion',
                   how='left').groupby('semana_epidemiologica_21').mean()

,n_folio
semana_epidemiologica_21,
62,92.960749
63,71.688138
64,68.519817
65,84.958797
66,84.492410
67,87.880471
68,85.364546
69,88.157462
70,91.361851


In [517]:
mean_ind5 = df_ind5_perc.reset_index().merge(data_aysen_clean_21[['fecha_notificacion','semana_epidemiologica_21']], 
                   left_on='fecha_cumple_requisitos',
                   right_on='fecha_notificacion',
                   how='left').groupby('semana_epidemiologica_21').mean()['n_folio'].mean()

In [518]:
px.scatter(df_ind5_perc.reset_index().merge(data_aysen_clean_21[['fecha_notificacion','semana_epidemiologica_21']], 
                   left_on='fecha_cumple_requisitos',
                   right_on='fecha_notificacion',
                   how='left').groupby('semana_epidemiologica_21').mean().reset_index(),
        x= 'semana_epidemiologica_21',
        y= 'n_folio',
        #trendline='rolling',
        #trendline_options=dict(window=5),
        color='n_folio',
        color_continuous_scale=px.colors.sequential.Viridis_r,
        labels={'n_folio':'Oportunidad (%)'},
        ).update_layout(title= 'Ind5: Oportunidad en la investigación epidemiológica de casos',
                       yaxis_title='Oportunidad (%)',
                       xaxis_title='Semana epidemiológica',
                       yaxis_range= [50,105],
                       template='simple_white',
                       xaxis=dict(tickmode = 'array',
                                  tickvals = list(range(12,88,3)),
                                  ticktext = list(list(range(12,54,3))+list(range(1,88,3))),
                                 ),
                    ).add_vline(x=53.5, line_width=3, line_dash='dash', line_color='gray'
                              ).add_hline(y=90,
                                            line_width=3, 
                                            line_dash='longdash', 
                                            line_color='#222A2A',
                                            annotation_text='Meta (90%)',
                                            annotation_position='bottom right'
                                # ).add_hline(y=mean_ind5, 
                                 #            line_width=3, 
                                  #           line_dash='longdash', 
                                   #          line_color='#0D2A63',
                                    #         annotation_text= 'Promedio ('+str(round(mean_ind5, 2))+'%)',
                                     #        annotation_position='bottom right'
                                     ).add_vrect(x0='51', x1='53', 
                                           annotation_text='2020', 
                                           annotation_position='inside right',
                                           annotation=dict(font_size=18, 
                                                           font_family="'Open Sans', verdana, arial, sans-serif"),
                                                           opacity=0, 
                                                           line_width=0
                                          ).add_vrect(x0='84', x1='88', 
                                           annotation_text='2021', 
                                           annotation_position='inside right',
                                           annotation=dict(font_size=18, 
                                                           font_family="'Open Sans', verdana, arial, sans-serif"),
                                                           opacity=0, 
                                                           line_width=0
                                                     ).update_traces(mode='lines+markers', 
                                                                     line_color='grey',
                                                                     marker_line_width=1, 
                                                                     marker_size=10)

# Indicador 6: Capacidad de investigación de contactos

## Objetivos: Fortalecer la investigación epidemiológica de los contactos estrechos

### Meta: >80%

**Periodicidad:** Semanal

**Escala territorial:** Comuna, Regional, Nacional

**Fuente de información:** base EPIVIGILA actualizada con los reportes de laboratorio al día de la determinación del indicador. Base de seguimiento

**Cálculo del indicador a nivel comunal**

$\frac{No.\:casos\:nuevos\:con\:contactos\:estrechos\:por\:comuna\:1er\:establecimiento\:semana\:x}{No.\:casos\:nuevos\:por\:comuna\:1er\:establecimiento\:semana\:x}$

**Cálculo del indicador a nivel regional**

$\frac{No.\:casos\:nuevos\:con\:contactos\:estrechos\:por\:región\:1er\:establecimiento\:semana\:x}{No.\:casos\:nuevos\:por\:región\:1er\:establecimiento\:semana\:x}$

**Observaciones**

- No se incluyen los casos nuevos no notificados
- Se considera un caso nuevo registrado a aquel caso nuevo notificado que cumple los requisitos para estar en el módulo de seguimiento
- Los casos nuevos con contactos estrechos son considerados de forma dicotómica (sí o no)
- El primer establecimiento corresponde al establecimiento de notificación. Se seguirá considerando el primer establecimiento para este indicador, aún si se derivó previo a la investigación epidemiológica.
- Se consideran sólo registros activos o egresados en la bandeja de seguimiento

In [519]:
df_seguimiento_ind6 = df_seguimiento_clean

In [520]:
df_seguimiento_ind6['is_contact']= df_seguimiento_ind6['nfolio_contacto_estrecho'].notna()

In [521]:
# Crea una lista con casos indice

casos_indice = df_seguimiento_ind6[df_seguimiento_ind6['nfolio_caso_indice'].notna()]['nfolio_caso_indice'].drop_duplicates().astype('float64').to_list()

casos_indice

[11425750.0,
 11714657.0,
 11964234.0,
 12293320.0,
 12436348.0,
 12350595.0,
 12608868.0,
 12861929.0,
 10587524.0,
 10848112.0,
 12340185.0,
 11685650.0,
 11223327.0,
 11688188.0,
 11691110.0,
 13701438.0,
 12468730.0,
 14062781.0,
 10436774.0,
 13083092.0,
 13690073.0,
 10883113.0,
 13357763.0,
 12228566.0,
 11981882.0,
 11215100.0,
 10824045.0,
 10884306.0,
 11137988.0,
 11619124.0,
 12606893.0,
 12522627.0,
 13210877.0,
 13962027.0,
 14062881.0,
 12679270.0,
 13872016.0,
 10073764.0,
 12442561.0,
 14074678.0,
 12800062.0,
 11561513.0,
 11476013.0,
 13921845.0,
 12146185.0,
 13691027.0,
 12142477.0,
 13120981.0,
 12433247.0,
 12787317.0,
 11902620.0,
 13848691.0,
 10781773.0,
 10782431.0,
 13923863.0,
 10285667.0,
 11131696.0,
 12041374.0,
 12207845.0,
 12052266.0,
 11637569.0,
 13919997.0,
 10839331.0,
 10440139.0,
 12224614.0,
 11558790.0,
 12787314.0,
 13648333.0,
 11266479.0,
 12492223.0,
 11718315.0,
 12882126.0,
 13797310.0,
 12793607.0,
 11682648.0,
 11902227.0,
 10618375.0,

In [522]:
# Revisa si un caso es caso indice

df_seguimiento_ind6['is_index'] = df_seguimiento_ind6['nfolio_caso'].isin(casos_indice)

In [523]:
# Crea columna para mostrando si es o caso indice o contacto estrecho

df_seguimiento_ind6['is_index_or_contact'] = df_seguimiento_ind6['is_index'] | df_seguimiento_ind6['is_contact']

In [524]:
df_seguimiento_ind6['is_index_or_contact'].value_counts()

False    119668
True      57890
Name: is_index_or_contact, dtype: int64

In [450]:
df_seguimiento_ind6_clean = df_seguimiento_ind6[(df_seguimiento_ind6['suspendido']=='activo')|(df_seguimiento_ind6['suspendido']=='egresado')]

In [451]:
# Elimina etapa_clinica DESCARTADA y SOSPECHA
df_seguimiento_ind6_clean = df_seguimiento_ind6_clean[(df_seguimiento_ind6_clean['etapa_clinica']!='DESCARTADA')|(df_seguimiento_ind6_clean['etapa_clinica']!='SOSPECHA')]

In [452]:
df_seguimiento_ind6_clean['nfolio_caso_indice'].value_counts()

19425664    22
15506064    22
13041090    22
13616788    21
15573170    21
            ..
17247820     1
17989525     1
14697163     1
13758599     1
13120153     1
Name: nfolio_caso_indice, Length: 13080, dtype: int64

In [453]:
df_seguimiento_ind6_clean.head(1)

,n_folio,nfolio_caso,nfolio_caso_indice,nfolio_pntm,nfolio_contacto_estrecho,tipo_seguimiento,dia_contacto,fecha_notificacion,nombre_institucion_indice,region_institucion_indice,...,comuna_seremi_xi,provincia_seremi_xi,edad,sexo,pop_hombres,pop_mujeres,pop_total,is_contact,is_index,is_index_or_contact
0,8753781,8753781.0,NaN,NaN,NaN,caso,2021-03-02 00:00:00,2021-03-01,Hospital Regional (Coihaique),11,...,Coyhaique,Coyhaique,47.0,hombre,28763.0,29055.0,57818.0,False,False,False


In [454]:
# Toma en cuenta solo el primer contacto con el contacto o caso

df_seguimiento_ind6_clean= df_seguimiento_ind6_clean.sort_values(by=['n_folio', 'dia_contacto']).groupby('n_folio').head(1)

In [455]:
df_seguimiento_ind6_clean.groupby(['fecha_cumple_requisitos', 'is_index_or_contact']).count()[['n_folio']].reset_index()

,fecha_cumple_requisitos,is_index_or_contact,n_folio
0,2021-03-01,False,14
1,2021-03-01,True,13
2,2021-03-02,False,21
3,2021-03-02,True,41
4,2021-03-03,False,10
...,...,...,...
361,2021-08-29,False,5
362,2021-08-29,True,15
363,2021-08-30,False,78
364,2021-08-30,True,4


In [456]:
# Crea gráfico de etapa_clinica regional por semana epidemiologica

px.bar(df_seguimiento_ind6_clean.groupby(['fecha_cumple_requisitos', 'is_index_or_contact']).count()[['n_folio']].reset_index(),
        x= 'fecha_cumple_requisitos',
        y= 'n_folio',
        color= 'is_index_or_contact',
        labels={'is_index_or_contact':'Contacto'}, 
       ).update_layout(title= 'Casos de la Región de Aysén con contactos estrechos',
                    yaxis_title='Número de casos',
                    xaxis_title='Fecha',
                    template='simple_white',
                      )                 
                    #xaxis=dict(tickmode = 'array',
                     #             tickvals = list(range(12,88,3)),
                      #            ticktext = list(list(range(12,54,3))+list(range(1,88,3))),
                      #           ),
               #     ).add_vline(x=53.5, line_width=3, line_dash='dash', line_color='gray'
                #               ).add_vrect(x0='50', x1='50', 
                 #                          annotation_text='2020', 
                  #                         annotation_position='top right',
                   #                        annotation=dict(font_size=18, 
                    #                                       font_family="'Open Sans', verdana, arial, sans-serif"),
                     #                                      opacity=0, 
                      #                                     line_width=0
                       #                   ).add_vrect(x0='84', x1='86', 
                        #                   annotation_text='2021', 
                         #                  annotation_position='top right',
                          #                 annotation=dict(font_size=18, 
                           #                                font_family="'Open Sans', verdana, arial, sans-serif"),
                            #                               opacity=0, 
                             #                              line_width=0)  
                      

In [457]:
df_ind6= df_seguimiento_ind6_clean.groupby(['fecha_cumple_requisitos', 'is_index_or_contact']).count()[['n_folio']].reset_index()

In [458]:
df_ind6_perc = (df_ind6[df_ind6['is_index_or_contact']==True].set_index('fecha_cumple_requisitos')[['n_folio']]/df_ind6.groupby('fecha_cumple_requisitos').sum('n_folio')[['n_folio']])*100

In [459]:
px.scatter(df_ind6_perc.reset_index(),
        x= 'fecha_cumple_requisitos',
        y= 'n_folio',
        color='n_folio',
        color_continuous_scale=px.colors.sequential.Magma_r,
        labels={'n_folio':'Oportunidad (%)'},
        ).update_layout(title= 'Capacidad de investigación de contactos estrechos',
                       yaxis_title='Oportunidad (%)',
                       xaxis_title='Fecha',
                       template='simple_white',
                 #      xaxis=dict(tickmode = 'array',
                  #                tickvals = list(range(12,88,3)),
                   #               ticktext = list(list(range(12,54,3))+list(range(1,88,3))),
                    #             ),
                 #   ).add_vline(x=53.5, line_width=3, line_dash='dash', line_color='gray'
                  #             ).add_hline(y=10, 
                   #                        line_width=3, 
                    #                       line_dash='longdash', 
                     #                      line_color='olive',
                      #                     annotation_text='Meta nacional (10%)',
                       #                    annotation_position='bottom right'
                        #         ).add_hline(y=mean_bac_ind, 
                         #                    line_width=3, 
                          #                   line_dash='longdash', 
                           #                  line_color='green',
                            #                 annotation_text= 'Promedio BAC ('+str(mean_bac_ind)+'%)',
                             #                annotation_position='bottom right'
                              #       ).add_vrect(x0='51', x1='50', 
                               #            annotation_text='2020', 
                                #           annotation_position='top right',
                                 #          annotation=dict(font_size=18, 
                                  #                         font_family="'Open Sans', verdana, arial, sans-serif"),
                                   #                        opacity=0, 
                                    #                       line_width=0
                                     #     ).add_vrect(x0='84', x1='86', 
                                      #     annotation_text='2021', 
                                       #    annotation_position='top right',
                                        #   annotation=dict(font_size=18, 
                                         #                  font_family="'Open Sans', verdana, arial, sans-serif"),
                                          #                 opacity=0, 
                                           #                line_width=0
                                                     ).update_traces(mode='lines+markers', 
                                                                     line_color='grey',
                                                                     marker_line_width=1, 
                                                                     marker_size=10)

In [460]:
mean_ind6 = df_ind6_perc.reset_index().merge(data_aysen_clean_21[['fecha_notificacion','semana_epidemiologica_21']], 
                   left_on='fecha_cumple_requisitos',
                   right_on='fecha_notificacion',
                   how='left').groupby('semana_epidemiologica_21').mean()['n_folio'].mean()

In [461]:
px.scatter(df_ind6_perc.reset_index().merge(data_aysen_clean_21[['fecha_notificacion','semana_epidemiologica_21']], 
                   left_on='fecha_cumple_requisitos',
                   right_on='fecha_notificacion',
                   how='left').groupby('semana_epidemiologica_21').mean().reset_index(),
        x= 'semana_epidemiologica_21',
        y= 'n_folio',
        #trendline='rolling',
        #trendline_options=dict(window=5),
        color='n_folio',
        color_continuous_scale=px.colors.sequential.Magma_r,
        labels={'n_folio':'Oportunidad (%)'},
        ).update_layout(title= 'Ind6: Capacidad de investigación de contactos',
                       yaxis_title='Oportunidad (%)',
                       xaxis_title='Semana epidemiológica',
                       yaxis_range= [0,100],
                       template='simple_white',
                       xaxis=dict(tickmode = 'array',
                                  tickvals = list(range(12,88,3)),
                                  ticktext = list(list(range(12,54,3))+list(range(1,88,3))),
                                 ),
                    ).add_vline(x=53.5, line_width=3, line_dash='dash', line_color='gray'
                              ).add_hline(y=80,
                                            line_width=3, 
                                            line_dash='longdash', 
                                            line_color='#222A2A',
                                            annotation_text='Meta (80%)',
                                            annotation_position='bottom right'
                                 ).add_hline(y=mean_ind6, 
                                             line_width=3, 
                                             line_dash='longdash', 
                                             line_color='#750D86',
                                             annotation_text= 'Promedio ('+str(round(mean_ind6, 2))+'%)',
                                             annotation_position='bottom right'
                                     ).add_vrect(x0='51', x1='53', 
                                           annotation_text='2020', 
                                           annotation_position='top right',
                                           annotation=dict(font_size=18, 
                                                           font_family="'Open Sans', verdana, arial, sans-serif"),
                                                           opacity=0, 
                                                           line_width=0
                                          ).add_vrect(x0='84', x1='88', 
                                           annotation_text='2021', 
                                           annotation_position='top right',
                                           annotation=dict(font_size=18, 
                                                           font_family="'Open Sans', verdana, arial, sans-serif"),
                                                           opacity=0, 
                                                           line_width=0
                                                     ).update_traces(mode='lines+markers', 
                                                                     line_color='grey',
                                                                     marker_line_width=1, 
                                                                     marker_size=10)

# Indicador 7: Capacidad de investigación de contactos

## Objetivo: Evaluar la capacidad de seguimiento de contactos

### Meta: >80%

**Periodicidad:** Semanal

**Escala territorial:** Comuna, Regional, Nacional

**Fuente de información:** base EPIVIGILA actualizada con los reportes de laboratorio al día de la determinación del indicador. Base de seguimiento.

**Cálculo del indicador a nivel comunal**

$\frac{No.\:contactos\:identificados\:investigados\:dentro\:de\:48hrs\:por\:comuna\:1er\:establecimiento\:semana\:x}{No.\:contactos\:identificados\:por\:comuna\:1er\:establecimiento\:semana\:x}x100$

**Cálculo del indicador a nivel regional**

$\frac{No.\:contactos\:identificados\:investigados\:dentro\:de\:48hrs\:por\:región\:1er\:establecimiento\:semana\:x}{No.\:contactos\:identificados\:por\:región\:1er\:establecimiento\:semana\:x}x100$

**Observaciones**

- No se incluyen los contactos nuevos no notificados
- Se considera como registro de contacto aquel que cumple las condiciones para estar en la bandeja de seguimiento
- Se consideran sólo registros de casos activos o egresados en la bandeja de seguimiento
- El primer establecimiento corresponde al establecimiento dado por API FONASA. Si el registro no hace match con API FONASA, se considera el establecimiento para este indicador, aún si se derivó previo a la investigación epidemiológica

In [462]:
df_seguimiento_ind7 = df_seguimiento_clean

In [463]:
df_seguimiento_ind7= df_seguimiento_ind7[df_seguimiento_ind7['nfolio_contacto_estrecho'].notna()]#[['fecha_cumple_requisitos','dia_contacto']]

In [464]:
# Filtra contactos que hayan sido localizados

df_seguimiento_ind7 = df_seguimiento_ind7[df_seguimiento_ind7['contacto_localizado']=='t']

In [465]:
# Se consideran sólo registros de casos activos o egresados
df_seguimiento_ind7_clean = df_seguimiento_ind7[(df_seguimiento_ind7['suspendido']=='activo')|(df_seguimiento_ind7['suspendido']=='egresado')]

In [466]:
df_seguimiento_ind7_clean = df_seguimiento_ind7_clean[df_seguimiento_ind7_clean['cont_estado_seguimiento']!='Aún no contactado']

In [467]:
df_seguimiento_ind7_clean = df_seguimiento_ind7_clean[df_seguimiento_ind7_clean['cont_inicio_cuarentena'].notna()]

In [468]:
df_seguimiento_ind7_clean= df_seguimiento_ind7_clean.sort_values(by=['n_folio', 'dia_contacto']).groupby('n_folio').head(1)

In [469]:
df_seguimiento_ind7_clean['dif_intervalo'] = pd.to_datetime(df_seguimiento_ind7_clean['fecha_cumple_requisitos']) - pd.to_datetime(df_seguimiento_ind7_clean['dia_contacto'])

In [470]:
df_seguimiento_ind7_clean['dif_intervalo'].value_counts()

0 days      11650
-1 days       304
-2 days       149
1 days        136
-3 days        88
-4 days        59
-5 days        55
3 days         40
2 days         22
-6 days        18
-7 days        13
-8 days        12
7 days         10
13 days         9
5 days          8
15 days         7
-13 days        5
27 days         3
6 days          3
29 days         3
-9 days         2
33 days         2
-10 days        2
28 days         1
8 days          1
4 days          1
16 days         1
-11 days        1
14 days         1
17 days         1
37 days         1
Name: dif_intervalo, dtype: int64

In [471]:
df_seguimiento_ind7_clean['dif_ind7'] = (df_seguimiento_ind7_clean['dif_intervalo'] <= pd.to_timedelta('2 days'))#| (df_seguimiento_ind7_clean['dif_intervalo'] == pd.to_timedelta('-1 days'))|(df_seguimiento_ind7_clean['dif_intervalo'] == pd.to_timedelta('0 days'))

In [472]:
df_seguimiento_ind7_clean['dif_ind7']

28191    True
37474    True
31378    True
32631    True
33965    True
         ... 
40617    True
38305    True
31220    True
30743    True
37593    True
Name: dif_ind7, Length: 12608, dtype: bool

In [473]:
df_seguimiento_ind7_clean.groupby(['fecha_cumple_requisitos','dif_ind7']).count()[['n_folio']].reset_index()

,fecha_cumple_requisitos,dif_ind7,n_folio
0,2021-03-01,True,13
1,2021-03-02,True,39
2,2021-03-03,True,46
3,2021-03-04,False,3
4,2021-03-04,True,24
...,...,...,...
205,2021-08-26,True,3
206,2021-08-27,True,5
207,2021-08-28,True,2
208,2021-08-29,True,15


In [474]:
# Crea gráfico de etapa_clinica regional por semana epidemiologica

px.bar(df_seguimiento_ind7_clean.groupby(['fecha_cumple_requisitos','dif_ind7']).count()[['n_folio']].reset_index(),
        x= 'fecha_cumple_requisitos',
        y= 'n_folio',
        color= 'dif_ind7',
        labels={'dif_ind7':'Cumplido'}, 
       ).update_layout(title= 'Capacidad de investigación de contactos en 48h',
                    yaxis_title='Número de casos',
                    xaxis_title='Fecha',
                    template='simple_white',
                      )                 
                    #xaxis=dict(tickmode = 'array',
                     #             tickvals = list(range(12,88,3)),
                      #            ticktext = list(list(range(12,54,3))+list(range(1,88,3))),
                      #           ),
               #     ).add_vline(x=53.5, line_width=3, line_dash='dash', line_color='gray'
                #               ).add_vrect(x0='50', x1='50', 
                 #                          annotation_text='2020', 
                  #                         annotation_position='top right',
                   #                        annotation=dict(font_size=18, 
                    #                                       font_family="'Open Sans', verdana, arial, sans-serif"),
                     #                                      opacity=0, 
                      #                                     line_width=0
                       #                   ).add_vrect(x0='84', x1='86', 
                        #                   annotation_text='2021', 
                         #                  annotation_position='top right',
                          #                 annotation=dict(font_size=18, 
                           #                                font_family="'Open Sans', verdana, arial, sans-serif"),
                            #                               opacity=0, 
                             #                              line_width=0)  
                      

In [475]:
df_ind7= df_seguimiento_ind7_clean.groupby(['fecha_cumple_requisitos', 'dif_ind7']).count()[['n_folio']].reset_index()

In [476]:
df_ind7_perc = (df_ind7[df_ind7['dif_ind7']==True].set_index('fecha_cumple_requisitos')[['n_folio']]/df_ind7.groupby('fecha_cumple_requisitos').sum('n_folio')[['n_folio']])*100

In [525]:
px.scatter(df_ind7_perc.reset_index(),
        x= 'fecha_cumple_requisitos',
        y= 'n_folio',
        color='n_folio',
        color_continuous_scale=px.colors.sequential.Mint,
        labels={'n_folio':'Oportunidad (%)'},
        ).update_layout(title= 'Capacidad de investigación de contactos en 48h',
                       yaxis_title='Oportunidad (%)',
                       xaxis_title='Fecha',
                       template='simple_white',
                 #      xaxis=dict(tickmode = 'array',
                  #                tickvals = list(range(12,88,3)),
                   #               ticktext = list(list(range(12,54,3))+list(range(1,88,3))),
                    #             ),
                 #   ).add_vline(x=53.5, line_width=3, line_dash='dash', line_color='gray'
                  #             ).add_hline(y=10, 
                   #                        line_width=3, 
                    #                       line_dash='longdash', 
                     #                      line_color='olive',
                      #                     annotation_text='Meta nacional (10%)',
                       #                    annotation_position='bottom right'
                        #         ).add_hline(y=mean_bac_ind, 
                         #                    line_width=3, 
                          #                   line_dash='longdash', 
                           #                  line_color='green',
                            #                 annotation_text= 'Promedio BAC ('+str(mean_bac_ind)+'%)',
                             #                annotation_position='bottom right'
                              #       ).add_vrect(x0='51', x1='50', 
                               #            annotation_text='2020', 
                                #           annotation_position='top right',
                                 #          annotation=dict(font_size=18, 
                                  #                         font_family="'Open Sans', verdana, arial, sans-serif"),
                                   #                        opacity=0, 
                                    #                       line_width=0
                                     #     ).add_vrect(x0='84', x1='86', 
                                      #     annotation_text='2021', 
                                       #    annotation_position='top right',
                                        #   annotation=dict(font_size=18, 
                                         #                  font_family="'Open Sans', verdana, arial, sans-serif"),
                                          #                 opacity=0, 
                                           #                line_width=0
                                                     ).update_traces(mode='lines+markers', 
                                                                     line_color='grey',
                                                                     marker_line_width=1, 
                                                                     marker_size=10)

In [478]:
mean_ind7 = df_ind7_perc.reset_index().merge(data_aysen_clean_21[['fecha_notificacion','semana_epidemiologica_21']], 
                   left_on='fecha_cumple_requisitos',
                   right_on='fecha_notificacion',
                   how='left').groupby('semana_epidemiologica_21').mean()['n_folio'].mean()

In [479]:
px.scatter(df_ind7_perc.reset_index().merge(data_aysen_clean_21[['fecha_notificacion','semana_epidemiologica_21']], 
                   left_on='fecha_cumple_requisitos',
                   right_on='fecha_notificacion',
                   how='left').groupby('semana_epidemiologica_21').mean().reset_index(),
        x= 'semana_epidemiologica_21',
        y= 'n_folio',
        #trendline='rolling',
        #trendline_options=dict(window=5),
        color='n_folio',
        color_continuous_scale=px.colors.sequential.Mint,
        labels={'n_folio':'Oportunidad (%)'},
        ).update_layout(title= 'Ind7: Capacidad de investigación de contactos en 48h',
                       yaxis_title='Oportunidad (%)',
                       xaxis_title='Semana epidemiológica',
                       yaxis_range= [0,105],
                       template='simple_white',
                       xaxis=dict(tickmode = 'array',
                                  tickvals = list(range(12,88,3)),
                                  ticktext = list(list(range(12,54,3))+list(range(1,88,3))),
                                 ),
                    ).add_vline(x=53.5, line_width=3, line_dash='dash', line_color='gray'
                              ).add_hline(y=80,
                                            line_width=3, 
                                            line_dash='longdash', 
                                            line_color='#222A2A',
                                            annotation_text='Meta (80%)',
                                            annotation_position='bottom left'
                                 ).add_hline(y=mean_ind7, 
                                             line_width=3, 
                                             line_dash='longdash', 
                                             line_color='#750D86',
                                             annotation_text= 'Promedio ('+str(round(mean_ind7, 2))+'%)',
                                             annotation_position='bottom left'
                                     ).add_vrect(x0='51', x1='53', 
                                           annotation_text='2020', 
                                           annotation_position='bottom right',
                                           annotation=dict(font_size=18, 
                                                           font_family="'Open Sans', verdana, arial, sans-serif"),
                                                           opacity=0, 
                                                           line_width=0
                                          ).add_vrect(x0='84', x1='88', 
                                           annotation_text='2021', 
                                           annotation_position='bottom right',
                                           annotation=dict(font_size=18, 
                                                           font_family="'Open Sans', verdana, arial, sans-serif"),
                                                           opacity=0, 
                                                           line_width=0
                                                     ).update_traces(mode='lines+markers', 
                                                                     line_color='grey',
                                                                     marker_line_width=1, 
                                                                     marker_size=10)

# Indicador 8: Casos en seguimiento

## Objetivo: Evaluar la capacidad de seguimiento de casos

### Meta: >90%

**Periodicidad:** Semanal

**Escala territorial:** Comuna, Regional, Nacional

**Fuente de Información:** base EPIVIGILA actualizada con los reportes de laboratorio al día de la determinación del indicador. Base de seguimiento

### Cálculo del indicador a nivel comunal

$\frac{No.\:casos\:activos\:investigados\:con\:seguimiento\:según\:protocolo\:por\:comuna\:último\:establecimiento\:semana\:x}{No.\:casos\:nuevos\:registrados\:por\:comuna\:ultimo\:establecimiento\:semana\:x}x100$

### Cálculo del indicador a nivel regional

$\frac{No.\:casos\:activos\:investigados\:con\:seguimiento\:según\:protocolo\:por\:región\:último\:establecimiento\:semana\:x}{No.\:casos\:nuevos\:registrados\:por\:región\:ultimo\:establecimiento\:semana\:x}x100$

**Observación**

- No se incluyen los casos nuevos no notificados ("casos reportados por laboratorio")
- Se considera un caso activo como aquel caso notificado que cumple la definición de caso activo, con fecha de notificación en la semana de interés
- Se consideran sólo registros activos o egresados en la bandeja de seguimiento
- Según protocolo implica un seguimiento diario por día de seguimineto, desde el inicio de seguimiento y se valora de forma dicotómica (cumple o no cumple el protocolo de seguimiento)
- Por seguimiento diario se entiende que diariamente, se debe realizar una actividad de seguimiento (la cual puede ser exitosa o no)

In [526]:
df_seguimiento_ind8 = df_seguimiento_clean

In [533]:
# Se consideran sólo registros de casos activos o egresados
df_seguimiento_ind8_clean = df_seguimiento_ind8[(df_seguimiento_ind8['suspendido']=='activo')|(df_seguimiento_ind8['suspendido']=='egresado')]

In [534]:
df_seguimiento_ind8_clean = df_seguimiento_ind8_clean[df_seguimiento_ind8_clean['cont_estado_seguimiento']!='Aún no contactado']

In [535]:
df_seguimiento_ind8_clean = df_seguimiento_ind8_clean[df_seguimiento_ind8_clean['cont_inicio_cuarentena'].notna()]

In [539]:
df_seguimiento_ind8_clean.sort_values(by=['n_folio', 'dia_contacto'])#.groupby('n_folio').count()

,n_folio,nfolio_caso,nfolio_caso_indice,nfolio_pntm,nfolio_contacto_estrecho,tipo_seguimiento,dia_contacto,fecha_notificacion,nombre_institucion_indice,region_institucion_indice,...,comuna_seremi_xi,provincia_seremi_xi,edad,sexo,pop_hombres,pop_mujeres,pop_total,is_contact,is_index,is_index_or_contact
29092,10000195,10000195.0,NaN,NaN,NaN,caso,2021-03-28 00:00:00,2021-03-26,Posta de Salud Rural Melinka,11,...,Guaitecas,Aysen,12.0,mujer,1166.0,677.0,1843.0,False,False,False
5608,10000195,10000195.0,NaN,NaN,NaN,caso,2021-03-30 00:00:00,2021-03-26,Posta de Salud Rural Melinka,11,...,Guaitecas,Aysen,12.0,mujer,1166.0,677.0,1843.0,False,False,False
41194,10000195,10000195.0,NaN,NaN,NaN,caso,2021-03-31 00:00:00,2021-03-26,Posta de Salud Rural Melinka,11,...,Guaitecas,Aysen,12.0,mujer,1166.0,677.0,1843.0,False,False,False
33928,10000195,10000195.0,NaN,NaN,NaN,caso,2021-04-01 00:00:00,2021-03-26,Posta de Salud Rural Melinka,11,...,Guaitecas,Aysen,12.0,mujer,1166.0,677.0,1843.0,False,False,False
36338,10000195,10000195.0,NaN,NaN,NaN,caso,2021-04-04 00:00:00,2021-03-26,Posta de Salud Rural Melinka,11,...,Guaitecas,Aysen,12.0,mujer,1166.0,677.0,1843.0,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29137,SC9998918-11899520,NaN,11899520,NaN,9998918,contacto,2021-04-01 00:00:00,2021-03-26,Posta de Salud Rural Melinka,11,...,Guaitecas,Aysen,57.0,mujer,1166.0,677.0,1843.0,True,False,True
40008,SC9998918-11899520,NaN,11899520,NaN,9998918,contacto,2021-04-02 00:00:00,2021-03-26,Posta de Salud Rural Melinka,11,...,Guaitecas,Aysen,57.0,mujer,1166.0,677.0,1843.0,True,False,True
30342,SC9998918-11899520,NaN,11899520,NaN,9998918,contacto,2021-04-03 00:00:00,2021-03-26,Posta de Salud Rural Melinka,11,...,Guaitecas,Aysen,57.0,mujer,1166.0,677.0,1843.0,True,False,True
33969,SC9998918-11899520,NaN,11899520,NaN,9998918,contacto,2021-04-04 00:00:00,2021-03-26,Posta de Salud Rural Melinka,11,...,Guaitecas,Aysen,57.0,mujer,1166.0,677.0,1843.0,True,False,True


# Indicador 10: Contactos con al menos 2 seguimientos dentro de su periodo de cuarentena de 14 días

## Objetivo: Evaluar la capacidad de seguimiento de contactos estrechos

### Meta: >90%

**Periodicidad:** Semanal

**Escala territorial:** Comuna, Regional, Nacional

**Fuente de información:** Base de seguimiento

### Cálculo del indicador a nivel comunal

$\frac{No.\:contactos\:que\:cumplieron\:cuarentena\:con\:al\:menos\:2\:seguimientos\:por\:comuna\:último\:establecimiento\:semana\:x}{No.\:contactos\:que\:cumplieron\:cuarentena\:por\:comuna\:ultimo\:establecimiento\:semana\:x}x100$

### Cálculo del indicador a nivel regional

$\frac{No.\:contactos\:que\:cumplieron\:cuarentena\:con\:al\:menos\:2\:seguimientos\:por\:región\:último\:establecimiento\:semana\:x}{No.\:contactos\:que\:cumplieron\:cuarentena\:por\:región\:ultimo\:establecimiento\:semana\:x}x100$

**Observación**

- El indicador se construye sobre los contactos que egresan (fecha de término de cuarentena) de seguimiento, durante la semana de interés (esto es, sólo registros egresados de la bandeja de seguimiento)
- El periodo de seguimiento para el contacto corresponde a la cantidad de días entre la 1era fecha en que el contacto está disponible en la bandeja de seguimiento hasta la última fecha fin cuarentena
- Se cumple con el indicador si en el período de seguimiento del contacto se registran al menos 2 actividades de seguimiento (las cuales pueden ser exitosas o no)
- Se excluyen los contactos que se transforman en caso y que están notificados (tiene folio asociado como caso)

In [527]:
df_seguimiento_ind10 = df_seguimiento_clean